In [21]:
import pandas as pd

In [22]:
gpus_dataset = pd.read_csv("Datasets/gpus.csv")

In [23]:
gpus_dataset.head()

,name,type,tdp_watts,TFLOPS32,TFLOPS16,GFLOPS32/W,GFLOPS16/W,memory,source
0,A100 PCIe 40/80GB,gpu,250,NaN,312.00,NaN,NaN,40.0,https://www.nvidia.com/content/dam/en-zz/Solut...
1,A100 SXM4 80 GB,gpu,400,NaN,312.00,NaN,NaN,80.0,https://www.nvidia.com/content/dam/en-zz/Solut...
2,AGX Xavier,gpu,30,16.00,32.00,533.33,1066.67,NaN,https://elinux.org/Jetson_AGX_Xavier
3,AMD RX480,gpu,150,5.80,5.80,38.67,38.67,8.0,techpowerup.com
4,GTX 1080 Ti,gpu,250,11.34,0.17,45.36,0.68,11.0,techpowerup.com


In [24]:
gpus_dataset = gpus_dataset[["name","type","tdp_watts"]]
gpu_dataset = gpus_dataset.sort_values("tdp_watts").reset_index(drop=True)
gpu_dataset.head()
gpu_dataset.to_csv("gpu_modified.csv",index=False)

In [25]:
gamma_dataset = pd.read_csv("Datasets/gamma.csv")
gamma_datasets = gamma_dataset.drop('Country code', axis=1)

In [26]:
idx = gamma_datasets.groupby('Country or region')['Year'].idxmax()
gamma_data = gamma_datasets.loc[idx]
gamma_data = gamma_data.rename(columns={'Country or region': 'Regions', 'Emissions intensity (gCO2 per kWh)': 'Gamma'})
gamma_data = gamma_data.sort_values("Gamma").reset_index(drop=True)
gamma_data.head()
gamma_data.to_csv("gamma_modified.csv",index=False)

In [27]:
pue_dataset = pd.read_csv("Datasets/pue.csv")
pue_dataset = pue_dataset.rename(columns={'Cloud Provider': 'Providers'})
pue_dataset = pue_dataset.sort_values("PUE").reset_index(drop=True)
pue_dataset.head()
pue_dataset.to_csv("pue_modified.csv",index=False)

In [28]:
from pandasql import sqldf

In [30]:
def data_parse(user_input,dataframe):
    query = ""
    try: 
        if dataframe == "gpu_dataset":
            query = "SELECT G.tdp_watts FROM gpus_dataset AS G WHERE G.name LIKE '{}'".format(user_input)
        elif dataframe == "gamma_data":
            query = "SELECT G.Gamma FROM gamma_data AS G WHERE G.Regions LIKE '{}'".format(user_input)
        elif dataframe == "pue_dataset":
            query = "SELECT G.PUE FROM pue_dataset AS G WHERE G.Providers LIKE '{}'".format(user_input)
        result = sqldf(query, env=None)
        resultant = float(result.iloc[0][0])
        return resultant
    except:
        print("The particular input is not available in the dataset")
        return None

In [36]:
result = data_parse("AGX Xavier","gpu_dataset")
print(type(result))

<class 'float'>


In [37]:
result2 = data_parse("Africa","gamma_data")
print(result2)

489.26


In [38]:
result2 = data_parse("Google Cloud","pue_dataset")
print(result2)

1.1


In [61]:
def calculate_carbon_emissions(num_of_chips,region, providers,grid_config, hours_trained, energy, num_processors, avg_power_per_processor,hardware_type):
    """
    Calculate carbon emissions using the formula: CF = γ · PUE · (power consumption*hours trained)
    energy : kwh
    gamma : kg/kwh
    CF: kg
    Args:
        PUE: Power Usage Effectiveness
        E_CPU: Energy consumption of CPU (optional)
        E_GPU: Energy consumption of GPU (optional)
        E_RAM: Energy consumption of RAM (optional)
        gamma: Gamma coefficient
        hours_trained: Hours of training
        num_processors: Number of processors
        avg_power_per_processor: Average power per processor per hour
    Returns:
        The calculated carbon emissions in kg.
    """
    total_energy = 0
    
    if energy:
        total_energy = energy
        
    else:
        val1 = data_parse(hardware_type,"gpu_dataset")
        if val1:
            total_energy = (hours_trained * val1 * num_of_chips)

        else:
            if num_processors and avg_power_per_processor:
                total_energy = hours_trained * num_processors * avg_power_per_processor

        total_energy = float(total_energy/1000) # kWh 
    val2 = data_parse(region,"gamma_data")
    if val2:
        gamma = val2
    else:
        gamma = 378.625  # g/Kwh
    gamma = float(gamma/1000) # kg/Kwh
    
    val3 = data_parse(providers,"pue_dataset")
    if val3:   # no units
        PUE = val3
    else:
        PUE = 1
    
    carbon_emissions = gamma * PUE * (total_energy)
    if grid_config and grid_config > 0:
        return carbon_emissions*grid_config
    else:
        return float(carbon_emissions/1000)


In [62]:
result = calculate_carbon_emissions(10000,"United States of America","Microsoft Azure",1,355.2,None,None,None,"Tesla V100-SXM2-32GB")
print(result)

300.0
1065600000.0
1065600.0
0.378625
1.18
476086.10399999993
